In [1]:
import quandl
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
import pytz
import sys
from boto3.session import Session
import boto3
import io
from io import StringIO
import botocore

In [2]:
quandl.ApiConfig.api_key = "sXju3Jbujdoy1eqBMaBy"

In [3]:
def get_pst_time():
    date_format='%Y-%m-%d %H:%M:%S %z'
    date = datetime.now(tz=pytz.utc)
    date = date.astimezone(timezone('US/Pacific'))
    pstDateTime=date.strftime(date_format)
    pstDateTime = "{0}:{1}".format(pstDateTime[:-2],pstDateTime[-2:])
    return pstDateTime

In [4]:
def save_s3_object(bucket,folder,filename,datafrane):
    csv_buffer = StringIO()
    datafrane.to_csv(csv_buffer, sep=",", index=True)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, folder+filename).put(Body=csv_buffer.getvalue()) 

In [5]:
def get_ndx():
    df = pd.read_csv('holdings_1625692286016.csv')
    return df['Holding Ticker'].str.strip().replace('.','_').replace('-','_')

In [6]:
def get_all_data(ticker):
    data = quandl.get(f"EOD/{ticker}")
    data.index = pd.to_datetime(data.index, utc=True)
    data.index = data.index.astype(str).str.replace('+', ' +')
    formattedTable = data[['Adj_Open', 'Adj_High', 'Adj_Low', 'Adj_Close', 'Adj_Volume']].copy()
    formattedTable['Engine'] = float("NaN")
    formattedTable['Source'] = "Quandl"
    formattedTable['Update'] = get_pst_time()
    return formattedTable.rename({'Adj_Open': 'Open', 'Adj_High' : 'High', 'Adj_Low' : 'Low', 'Adj_Close' : 'Close', 'Adj_Volume' : 'Volume'}, axis=1)

In [7]:
#sys.stdout = open('/dev/stdout', 'w')
ndxList = get_ndx()
counter = 0
total = ndxList.shape[0]
for ticker in ndxList:
    counter += 1
    print(f"{ticker}: {counter}/{total}")
    df = get_all_data(ticker)
    save_s3_object('lc-market-data', 'Equities/US/', f"{ticker}_daily.txt", df)

AAPL: 1/102
MSFT: 2/102
AMZN: 3/102
FB: 4/102
GOOG: 5/102
TSLA: 6/102
NVDA: 7/102
GOOGL: 8/102
PYPL: 9/102
ADBE: 10/102
CMCSA: 11/102
NFLX: 12/102
INTC: 13/102
CSCO: 14/102
PEP: 15/102
AVGO: 16/102
TMUS: 17/102
TXN: 18/102
COST: 19/102
QCOM: 20/102
AMGN: 21/102
CHTR: 22/102
INTU: 23/102
SBUX: 24/102
AMAT: 25/102
AMD: 26/102
ISRG: 27/102
ZM: 28/102
MRNA: 29/102
BKNG: 30/102
MU: 31/102
LRCX: 32/102
MDLZ: 33/102
GILD: 34/102
ADP: 35/102
MELI: 36/102
CSX: 37/102
ATVI: 38/102
FISV: 39/102
ILMN: 40/102
ADSK: 41/102
ADI: 42/102
REGN: 43/102
JD: 44/102
IDXX: 45/102
DOCU: 46/102
NXPI: 47/102
ASML: 48/102
BIIB: 49/102
VRTX: 50/102
KDP: 51/102
KHC: 52/102
BIDU: 53/102
MNST: 54/102
ALGN: 55/102
KLAC: 56/102
MRVL: 57/102
EBAY: 58/102
LULU: 59/102
MAR: 60/102
ROST: 61/102
WDAY: 62/102
EXC: 63/102
MTCH: 64/102
PDD: 65/102
DXCM: 66/102
SNPS: 67/102
AEP: 68/102
WBA: 69/102
EA: 70/102
ALXN: 71/102
ORLY: 72/102
CTAS: 73/102
MCHP: 74/102
PAYX: 75/102
CDNS: 76/102
TEAM: 77/102
CTSH: 78/102
XEL: 79/102
XLNX